In [ ]:
options = Options()
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
 
review_list = []
 
for title in total_title_list[:10]:
    url = f"{title}"
    driver.get(url)
    try:
        tap = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#tap_moving_2 > a")))
        tap.click()
    except (NoSuchElementException, TimeoutException):
        print(f"{title} 에는 리뷰 섹션이 없습니다. 건너뜁니다.")
        continue  
    while True:
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "div.box__content > div.box__review-text > p")))
            reviews = driver.find_elements(By.CSS_SELECTOR, "div.box__content > div.box__review-text > p")
            if len(reviews) == 0:
                continue
            for review in reviews:
                review_text = review.text.strip()
                if review_text:
                    review_list.append(review_text)
            next_buttons = driver.find_elements(By.CSS_SELECTOR, "#divVipReview > div > a.link__page-move.link__page-next")
            if not next_buttons:
                print(f"{title}의 리뷰가 더 이상 없습니다.")
                break  
            next_button = next_buttons[0]
            if 'disabled' in next_button.get_attribute('class'):
                print(f"{title}의 마지막 페이지에 도달했습니다.")
                break 
            try:
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#divVipReview > div > a.link__page-move.link__page-next")))
                next_button.click()
                print("다음 페이지로 이동합니다.")
                WebDriverWait(driver, 10).until(
                    EC.staleness_of(next_button))
            except (StaleElementReferenceException, ElementClickInterceptedException) as e:
                print(f"다음 페이지 버튼 클릭 중 오류 발생: {e}. 다시 시도합니다.")
                continue  
        except (NoSuchElementException, TimeoutException):
            print(f"{title}의 리뷰를 더 이상 가져올 수 없습니다.")
            break  
        except StaleElementReferenceException:
            print("요소가 유효하지 않습니다. 다시 시도합니다.")
            break  
 
driver.quit()
print(review_list)